In [36]:
import pandas as pd
import numpy as np 
import string

def isEnglish(s):
   try:
       s.encode('ascii')
       return 1
   except:
       return 0

data = pd.read_csv("json_to_csv/out.csv")
data.columns
print(data.shape)

data = data[(data['educations-field_of_study'] != "--undefined--") & (data['industry'] != "--undefined--")]

columns = ["industry", "educations-degree", "educations-field_of_study", "educations-school-name", "positions-is-current", "num-connections"]
new_data = data[columns]
new_data['positions-is-current'] = new_data['positions-is-current'].replace('--undefined--', 'FALSE')
new_data = new_data.replace("--undefined--",np.NaN)
new_data = new_data.reset_index()

#print(new_data.shape[0])
is_english = []
for i in range(0, new_data.shape[0]):
   is_english.append(isEnglish(new_data["industry"][i]))
new_data["is_english"] = is_english

new_data[(new_data["is_english"] == 0) & (pd.isnull(new_data["industry"])==False)]
new_data

for i in columns:
    new_data[i] = [str(j).upper() for j in new_data[i].tolist()]
    
new_data

(888664, 16)


C:\Users\ara\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,index,industry,educations-degree,educations-field_of_study,educations-school-name,positions-is-current,num-connections,is_english
0,24,INFORMATION TECHNOLOGY AND SERVICES,BE,ELECTRONICS AND COMMUNICATION,THE MADRAS SEVA SADAN HIGHER SECONDARY SCHOOL,FALSE,1,1
1,39,EDUCATION MANAGEMENT,MASTER OF BUSINESS ADMINISTRATION (M.B.A.),MARKETING,MUMBAI UNIVERSITY,FALSE,13,1
2,45,EDUCATION MANAGEMENT,ISLAMIC EDUCATION,ISLAMIC EDUCATION,UNIVERSITI MALAYA,TRUE,0,1
3,53,PUBLIC RELATIONS AND COMMUNICATIONS,MASTER'S,PUBLIC COMMUNICATION,INSTITUT D'ETUDES POLITIQUES DE LILLE,FALSE,112,1
4,58,AUTOMOTIVE,MCA,COMPUTER APPLICATIONS,ST. JOSEPH COLLEGE CUDDALORE,FALSE,18,1
5,59,FINANCIAL SERVICES,PGDMM,MARKETING,ANNAMALAI UNIVERSITY,TRUE,10,1
6,62,COMPUTER SOFTWARE,B.TECH,INFORMATION TECHNOLOGY,ANNA UNIVERSITY,TRUE,56,1
7,71,ACCOUNTING,B.COM,COMMERCE,KERALA UNIVERSITY,FALSE,2,1
8,113,"LEISURE, TRAVEL & TOURISM",BSC,"PHYSICS,CHEMISTRY,MATHS",MANGALORE UNIVERSITY,FALSE,74,1
9,130,INFORMATION TECHNOLOGY AND SERVICES,MASTER OF TECHNOLOGY (M.TECH.),COMPUTER SCIENCE,VISVESVARAYA TECHNOLOGICAL UNIVERSITY,FALSE,75,1
